# Project1 PNLF Postproduction <a class="tocSkip">
    
After running the production notebook, this notebook can be used to create shared plots for the galaxies and LaTeX output tables

## Preparation
     
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pymuse.packages import *

from pymuse.constants import tab10, single_column, two_column

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,format='%(levelname)s: %(message)s',level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [ ]:
# first we need to specify the path to the raw data
basedir = Path('..')
data_raw = basedir / 'data' / 'raw' / 'MUSE' / 'DR2'

with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)
    
result = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
result.add_index('name')
result.rename_columns(['mu','mu+','mu-','mu_SNR','mu_SNR+','mu_SNR-'],['dis','dis_plus','dis_minus','dis_SNR','dis_SNR_plus','dis_SNR_minus'])

with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)    

## LaTeX sample table

In [ ]:
from pymuse.auxiliary import search_table

def area(dic):
    '''Calculate the area in an image
    
    from the number of pixels and the distance,
    corrected for inclination
    '''
    
    size_of_pixel = 0.2*u.arcsec

    distance = Distance(distmod=dic['mu'])
    pixel_area = (size_of_pixel/u.arcsec * distance/206265)**2
        
    return pixel_area.to(u.kpc**2) *dic['Npixel'] / np.cos(dic['inclination']*u.deg)


filename = basedir / 'data' / 'external' / 'phangs_sample_table_v1p5.fits'
with fits.open(filename) as hdul:
    sample_table = Table(hdul[1].data)

#galaxies = sample_table[sample_table['HAS_MUSE']==1]['NAME']
galaxies = sample_table[sample_table['survey_muse_status']!='not_in_survey']['name']

latexdict = {'tabletype': 'table*',
'header_start': '\\toprule\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Galaxy sample',
'units': {'R.A.':'(J2000)','Dec.':'(J2000)','Inclination':'deg','Distance':'$\si{\mega\parsec}$',
          'Survey Area':'$\si{\square\kilo\parsec}$'},
'preamble': '\\centering',
'tablefoot': f'\\label{{tbl:sample}}'
            }
 
sample_dict = {
'Name': [],
'Type':[],
'R.A.': [],
'Dec.': [],
'Distance': [],
'Inclination': [],
'mass': [],
'SFR': [],
'Survey Area': []
}

    
for name in galaxies:
    stbl = search_table(sample_table,name)
    d = stbl["dist"][0]
    d_unc = stbl["dist_unc"][0]

    sample_dict['Type'].append(stbl['morph_string'])    
    sample_dict['Name'].append(name.upper())
    sample_dict['R.A.'].append(stbl['orient_ra'])
    sample_dict['Dec.'].append(stbl['orient_dec'])
    sample_dict['Distance'].append(f'{d:.2f}$\pm${d_unc:.2f}')
    sample_dict['Inclination'].append(stbl['orient_incl'])
    sample_dict['mass'].append(stbl['props_mstar'])
    sample_dict['SFR'].append(stbl['props_sfr'])
    try:
        sample_dict['Survey Area'].append(f'{area(parameters[name.upper()]).value:.1f}')
    except:
        sample_dict['Survey Area'].append('NaN')
    '''
    sample_dict['Name'].append(name)
    sample_dict['R.A.'].append(stbl['ORIENT_RA'])
    sample_dict['Dec.'].append(stbl['ORIENT_DEC'])
    sample_dict['Distance'].append(stbl['DIST'])
    sample_dict['Inclination'].append(stbl['ORIENT_INCL'])
    sample_dict['$\log_{10}(M_*)$'].append(stbl['MSTAR_LOGMSTAR'])
    sample_dict['$\log_{10}($SFR$)$'].append(stbl['SFR_LOGSFR'])
    '''

sample = Table(sample_dict)
sample['mass'] = np.log10(sample['mass'])
sample['SFR'] = np.log10(sample['SFR'])
sample['mass'].info.format = '%.2f' 
sample['SFR'].info.format = '%.2f' 
coord = SkyCoord(sample['R.A.']*u.degree,sample['Dec.']*u.degree)
sample['R.A.'], sample['Dec.'] = zip(*[x[0].split(' ') for x in coord.to_string(style='hmsdms',precision=2)])
#sample.add_column('',index=1,name='Type')

with open(basedir / 'data' / 'catalogues' / 'sample.txt','w',newline='\n') as f:
    ascii.write(sample,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')
    
sample.rename_columns(['mass','SFR'],['$\log_{10}(M_*/\si{\Msun})$','$\log_{10}($SFR$/\si{\Msun \per \year})$'])
with open(basedir / 'data' / 'catalogues' /'sample.tex','w',newline='\n') as f:
    ascii.write(sample,f,Writer=ascii.Latex, latexdict=latexdict,overwrite=True)
    
#sample

## LaTeX result table

In [ ]:
result['dis_Mpc'] = Distance(distmod=result['dis']).to(u.Mpc)
result['dis_Mpc_plus'] = (Distance(distmod=result['dis']+result['dis_plus']) - result['dis_Mpc']).to(u.Mpc)
result['dis_Mpc_minus'] = (result['dis_Mpc']-Distance(distmod=result['dis']-result['dis_minus'])).to(u.Mpc)

result['dis_SNR_Mpc'] = (Distance(distmod=result['dis_SNR'])).to(u.Mpc)
result['dis_SNR_Mpc_plus'] = (Distance(distmod=result['dis_SNR']+result['dis_SNR_plus']) - result['dis_SNR_Mpc']).to(u.Mpc)
result['dis_SNR_Mpc_minus'] = (result['dis_SNR_Mpc']-Distance(distmod=result['dis_SNR']-result['dis_SNR_minus'])).to(u.Mpc)

In [ ]:
# save to file
for col in ['dis','dis_plus','dis_minus','dis_Mpc','dis_Mpc_plus','dis_Mpc_minus']:
    result[col].info.format = '%.2f'

distance_modulus = []
distance_parsec = []
for row in result:
    distance_modulus.append(f'{row["dis"]:.2f} + {row["dis_plus"]:.2f} - {row["dis_plus"]:.2f}')
    distance_parsec.append(f'{row["dis_Mpc"]:.2f} + {row["dis_Mpc_plus"]:.2f} - {row["dis_Mpc_plus"]:.2f}')
result['mu'] = distance_modulus
result['d/Mpc'] = distance_parsec
                           
filename = basedir / 'data' / 'catalogues' / f'all_objects.txt'
with open(filename,'w',newline='\n') as f:
    ascii.write(result[['name','mu','d/Mpc']],
                f,format='fixed_width',delimiter='\t',overwrite=True)

In [ ]:
tbl_out = ''
result.sort('name')
for row in result:
    tbl_out += f'{row["name"]} & {row["N_PN"]} & {row["N_SNR"]} ({row["N_SNRorPN"]}) '
    tbl_out += f'& $\\uncertainty{{{row["dis"]:.2f}}}{{{row["dis_plus"]:.2f}}}{{{row["dis_minus"]:.2f}}}$ '
    tbl_out += f'& $\\uncertainty{{{row["dis_Mpc"]:.2f}}}{{{row["dis_Mpc_plus"]:.2f}}}{{{row["dis_Mpc_minus"]:.2f}}}$ '
    tbl_out += f'& $\\uncertainty{{{row["dis_SNR"]:.2f}}}{{{row["dis_SNR_plus"]:.2f}}}{{{row["dis_SNR_minus"]:.2f}}}$ '
    tbl_out += f'& $\\uncertainty{{{row["dis_SNR_Mpc"]:.2f}}}{{{row["dis_SNR_Mpc_plus"]:.2f}}}{{{row["dis_SNR_Mpc_minus"]:.2f}}}$\\\\\n'
    
    
print(tbl_out)    

In [ ]:
def uncertainties(mu,mu_plus,mu_minus):
    d       = Distance(distmod=mu)
    d_plus  = Distance(distmod=mu+mu_plus) - Distance(distmod=mu)
    d_minus = Distance(distmod=mu)-Distance(distmod=mu-mu_minus)
    
    return d,d_plus,d_minus

## Combined PNLF

In [ ]:
# here we define possible lists of objects to plot
DR2  = ['NGC1300','NGC1385','NGC1433','NGC4303','NGC4321','NGC7496']
good = ['IC5332','NGC0628','NGC1566','NGC3351','NGC3627','NGC5068']
PHANGS_sample = ['IC5332','NGC1087','NGC1300','NGC1365','NGC1385','NGC1433', 'NGC1512','NGC1566','NGC1672',
                 'NGC2835','NGC3351','NGC3627','NGC4254','NGC4303','NGC4321','NGC4535','NGC5068','NGC7496']

In [ ]:
from pymuse.plot.pnlf import _plot_pnlf

names = PHANGS_sample[12:]
nrows = 2
ncols = 3
filename = basedir / 'reports' / f'all_objects2_PNLF'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects') 
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(two_column,two_column/ncols*nrows))
axes = iter(axes_arr.flatten())
Mmax = -4.47
    
# loop over the galaxies we want to plot
for name in names:  
    # read in the data
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis and find position on the grid
    ax = next(axes)
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    # pre-process the data for the plot and read additional parameters
    data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['mOIII']
    mu = result.loc[name]['dis']
    completeness = parameters[name]['completeness_limit']
    cut = parameters[name].get('cut',0)
    if cut>0:
        print(f'warning: using cut={cut} for {name}')
    data = data[data>cut]
    binsize = parameters[name]['binsize']
    mlow = Mmax+mu
    mhigh = 30

    ax=_plot_pnlf(data,mu,completeness,binsize=binsize,ax=ax)

    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'N')
    ax.set_title(name)
    
plt.tight_layout()
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## Combined cumulative PNLF

In [ ]:
from pymuse.plot.pnlf import _plot_cum_pnlf

names = PHANGS_sample[6:]
nrows = 4
ncols = 3
filename = basedir / 'reports' / f'all_objects_PNLF_cum'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
if nrows*ncols<len(names):
    raise ValueError('not enough subplots for selected objects')
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())

binsize=0.1
Mmax = -4.47
color = 'tab:red'

for name in names:  
        
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
    else:
        print(f'no catalogue for {name}')
        continue
    
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    data = catalogue[(catalogue['type']=='PN') & (~catalogue['exclude'])]['mOIII']
    mu = result.loc[name]['dis']
    completeness = parameters[name]['completeness_limit']
    cut = parameters[name].get('cut',0)
    if cut>0:
        print(f'warning: using cut={cut} for {name}')
    data = data[data>cut]
    
    ax=_plot_cum_pnlf(data,mu,completeness,ax=ax,binsize=0.1)
    
    # add labels to the axis
    if i==nrows-1:
        ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    if j==0:
        ax.set_ylabel(r'Cumulative N')
    
    ax.set_title(name)
    
plt.tight_layout()
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.show()

## Combine line diagnostics

In [ ]:
from pymuse.analyse import PNLF

names = result['name']
style = {'SNR':{"marker":'o',"ms":3,"mfc":'None',"mec":tab10[0],'ls':'none','ecolor':tab10[0]},
         'SNRorPN':{"marker":'o',"ms":4,"mfc":'white',"mec":'tab:green','ls':'none','ecolor':'tab:green'},
         'HII':{"marker":'+',"ms":3,"mec":tab10[1],'ls':'none'},
         'PN':{"marker":'o',"ms":2,"mfc":'black','mec':'black','ls':'none','ecolor':'black'}
        }
Mmax=-4.47
color = 'tab:red'

# define the figure with the number of subplots
nrows = 2
ncols = 3
width = two_column
fig, axes_arr = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes = iter(axes_arr.flatten())


names = ['IC5332','NGC0628','NGC1566','NGC3351','NGC3627','NGC5068']

# loop over the galaxies we want to plot
for name in names:  
       
    # read in the data
    catalogue_file = basedir / 'data' / 'catalogues' / f'{name}_nebulae.txt'
    if catalogue_file.is_file():
        catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
        catalogue['exclude'] = catalogue['exclude'].astype(bool)
        for col in catalogue.columns:
            if col.endswith('detection'):
                catalogue[col] = catalogue[col]=='True'
    else:
        print(f'no catalogue for {name}')
        continue
        
    # get the next axis
    ax = next(axes)
    # find current position on the grid
    i, j = np.where(axes_arr == ax)
    i,j=i[0],j[0]
    
    # pre-process the data for the plot and read additional parameters
    mu = result.loc[name]['dis']
    completeness = parameters[name]['completeness_limit']
    
    # draw line that we use seperate PN from HII
    MOIII = np.linspace(-5,1)
    OIII_Ha = 10**(-0.37*(MOIII)-1.16)
    ax.plot(MOIII,OIII_Ha,c='black',lw=0.6)
    ax.axhline(10**4)

    if completeness:
        ax.axvline(completeness-mu,ls='--',c='grey',lw=0.5)
    ax.axvline(Mmax,ls='--',c='grey',lw=0.5)

    for t in ['HII','PN','SNR']:
        tbl = catalogue[catalogue['type']==t]        
        ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),**style[t],label=t) 

        if t=='PN':
            # indicate for which PN we don't have a detection in HA6562
            tbl = tbl[~tbl['HA6562_detection']]
            ax.errorbar(tbl['mOIII']-mu,1.11*tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),
                         marker=r'$\uparrow$',ms=4,mec='black',ls='none') 
        if t=='SNR':
            #tbl = tbl[tbl['SNRorPN']] 
            ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']), marker='x',ms=2,mec=tab10[0],ls='none') 
   
    # objects that were rejeceted by eye
    tbl = catalogue[catalogue['exclude']]
    ax.errorbar(tbl['mOIII']-mu,tbl['OIII5006']/(tbl['HA6562']+tbl['NII6583']),marker='o',ms=3,ls='none',color='tab:green') 
    
    
    # configure axes 
    ax.set(xlim=[-5,np.ceil(completeness-mu)],
           ylim=[0.03,200],
           yscale='log')
    
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.16g}'.format(y)))

    axt = ax.twiny()
    xlim1,xlim2 = ax.get_xlim()
    axt.set_xticks(np.arange(np.ceil(xlim1+mu),np.floor(xlim2+mu)+1),minor=False)
    axt.set(xlim   = [xlim1+mu,xlim2+mu])
    
    if i==0:
        axt.set(xlabel = r'$m_{\mathrm{[OIII]}}$')
    
    if i==nrows-1:
        ax.set(xlabel=r'$M_{\mathrm{[OIII]}}$')
    if j==0:
        ax.set(ylabel=r'[OIII] / $(\mathrm{H}\alpha + \mathrm{[NII]})$')
    ax.set_title(name)
    
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    
plt.tight_layout()
filename = basedir / 'reports' / f'all_objects_line_diagnostics'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.savefig(filename.with_suffix('.png'),bbox_inches='tight',dpi=600)

plt.show()

## PN Number vs Mass etc.

In [ ]:
sample=ascii.read(basedir/'reports'/'sample.txt')
sample['SkyCoord'] = SkyCoord(sample['R.A.'],sample['Dec.'])
sample.add_index('Name')

In [ ]:
galaxies = {}

for n in result['name']:
    print('are you sure that you want to run this?')
    break
    filename = data_raw / 'MUSEDAP' / f'{n}_MAPS.fits'

    with fits.open(filename) as hdul:
        d=hdul['STELLAR_MASS_DENSITY'].data
        galaxies[n]= np.nansum(d)

In [ ]:
result['mass']=sample['mass']
result['Survey Area'] = sample['Survey Area']

Calculate the survey area from number of pixels and distance

In [ ]:
for row in result:
    N_pix = row['N_pixel']
    d = Distance(distmod=parameters[row['name']]['mu'])
    A_pix = ((d*(0.2/206265))**2).to(u.kpc**2)
    
    print(f'{row["name"]}: {d:.2f}, {N_pix*A_pix:.2f}')

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,nrows=1,figsize=(12,4))

for row in result:
    if row['N_PN']>1:
        ax1.scatter(row['mass'],row['N_PN'])
        ax1.text(row['mass'],row['N_PN']+2,row['name'],horizontalalignment='center')
        
        
        ax2.scatter(row['Survey Area'],row['N_PN'])
        ax2.text(row['Survey Area'],row['N_PN']+2,row['name'],horizontalalignment='center') 

        ax3.scatter(row['N_pixel'],row['N_PN'])
        ax3.text(row['N_pixel'],row['N_PN']+2,row['name'],horizontalalignment='center') 
        

ax1.set(xlabel='stellar mass density',ylabel='N PN')
ax2.set(xlabel='Survey Area')
ax3.set(xlabel='Npixel')
#ax3.set(xlabel='Npixel')
plt.show()

## Other PNLF studies

In [ ]:
distances = ascii.read(basedir/'data'/'literature distances'/'latest.csv',delimiter=',',header_start=12,data_start=14)
#distances = ascii.read(basedir/'data'/'literature distances'/'NED-D-16.1.0.csv',delimiter=',',header_start=12,data_start=14)
results = ascii.read(basedir/'data'/'interim'/'results.txt')

In [ ]:
pnlf_distances = distances[distances["Method"]=='PNLF']
local_group = Distance(3*u.Mpc).distmod
pnlf_distances = pnlf_distances[(pnlf_distances['m-M']>22.5) & (pnlf_distances['m-M']<32)]  # exclude the many measurments of the LMC and SMC
pnlf_distances['year'] = pnlf_distances['Date (Yr. - 1980)']+1980
pnlf_distances.rename_column('Galaxy ID','name')
pnlf_distances['name'] = [n.rstrip('a').rstrip('b') for n in pnlf_distances['name']]

alias = {
    'NGC 0628': 'MESSIER 074',
    'NGC 3351': 'MESSIER 095',
    'NGC 3627': 'MESSIER 066',
    'NGC 4254': 'MESSIER 099',
    'NGC 4303': 'MESSIER 061',
    'NGC 4321': 'MESSIER 100'
}

phangs_sample = []
for row in results:
    name = row['name'].replace('NGC','NGC ').replace('IC','IC ')
    name = alias.get(name,name)
    phangs_sample.append(name)
    new = ['',0,0,name,row['mu'],row['mu-'],0,'PNLF','Schmnn+2020','',0,0,0,40,'',2020]
    pnlf_distances.add_row(new)
    
galaxies = list(np.unique(pnlf_distances['name']))

In [ ]:
# we sort by mean distance
mean_dis = []
for gal in galaxies:
    mean_dis.append(pnlf_distances[pnlf_distances['name']==gal]['m-M'].mean())

In [ ]:

fig, ax = plt.subplots(1,figsize=(10,4))

x_pos  = []
gal_names = [x for _,x in sorted(zip(mean_dis,galaxies))]

print(f'{len(gal_names)} galaxies in sample')
for i,gal in enumerate(gal_names):
    tmp = pnlf_distances[pnlf_distances['name']==gal]
    if tmp['m-M'][0]>0:
        ax.scatter(len(tmp)*[i+1],tmp['m-M'],marker="_",color='black')
        
        # color the measured by me red
        if len(tmp[tmp['REFCODE']=='Schmnn+2020'])>0:
            ax.scatter([i+1],tmp[tmp['REFCODE']=='Schmnn+2020']['m-M'],marker="_",color='tab:red')


# create a legend with numbers only
#legend_elements = [mpl.lines.Line2D([0], [0], color=tab10[i], lw=2, label=str(i+1)) for i in range(10)]
#plt.legend(handles=legend_elements, loc='upper center',ncol=10)
    
ymin,ymax = 23,32
# set the galaxy names as x-ticklabels
ax.set(xticks=np.arange(1,len(galaxies)+1),
       ylabel='($m-M$) / mag',
       title='Galaxies with PNLF distances',
       xlim=[0.5,len(galaxies)+0.5],
       ylim=[ymin,ymax])
ax.set_xticklabels(gal_names,rotation=90)    

# color the galaxies which are in the phangs sample 
for n in phangs_sample:
    i = gal_names.index(n)
    ax.get_xticklabels()[i].set_color("tab:red")
#for n in ['MESSIER 066','MESSIER 074','MESSIER 095','NGC 5068']:
#    i = gal_names.index(n)
#    ax.get_xticklabels()[i].set_color("tab:red")    
ax.grid(axis='x')

yticks_mpc = np.logspace(np.log10(Distance(distmod=ymin).to(u.Mpc).value),np.log10(Distance(distmod=ymax).to(u.Mpc).value),10)
yticks_mu  = Distance(yticks_mpc*u.Mpc).distmod
    
ax2 = ax.twinx()
ax2.set_yticks(yticks_mu.value,minor=False)
ax2.set_yticklabels([f'{x:.2f}' for x in yticks_mpc],ha="left")
ax2.set(ylim=[ymin,ymax],ylabel='$D$ / Mpc')

plt.tight_layout()

plt.savefig(basedir/'reports'/'PNstudies.png',dpi=600)
plt.show()

In [ ]:
from scipy.stats import mode

In [ ]:
def mode_only(array):
    m,c = mode(array)
    return m

In [ ]:
pnlf_distances[pnlf_distances['name']=='MESSIER 066']

In [ ]:
groups = pnlf_distances.group_by('REFCODE')
plt.hist(groups.groups.aggregate(mode_only)['year'],bins=np.arange(1980,2020,1))

plt.show()

In [ ]:
deep = ascii.read(basedir/'data'/'literature distances'/'deep_distances.csv')
deep.add_index("galaxy")
deep['d'] = [float(x[:-4]) for x in deep['distance']]
deep['e'] = [float(x[:-4]) for x in deep['error']]

In [ ]:
fig,ax = plt.subplots(figsize=(two_column,two_column/1.618))
result.sort('dis_Mpc')
for i, row in enumerate(result):
    ax.errorbar(i-0.1,row['dis_Mpc'],yerr=row['dis_Mpc_minus'],fmt='o',color='tab:red')
    
    tmp = deep.loc[row['name']]
    ax.errorbar(i+0.1,tmp['d'],yerr=tmp['e'],fmt='o',color='black')
    
ax.set(xticks=np.arange(0,len(result)),
       ylabel='$D$ / Mpc',
       title='PHANGS distances',
       xlim=[-0.5,len(result)-0.5])
ax.set_xticklabels(result['name'],rotation=90)  
ax.grid(axis='x')

legend_elements = [mpl.lines.Line2D([0], [0], color=col, lw=2, label=l) for col,l in zip(['tab:red','black'],['PNLF','TRGB'])]
plt.legend(handles=legend_elements, loc='lower center',ncol=10)
plt.savefig(basedir/'reports'/'PNLF_vs_TRGB.png',dpi=600)

plt.show()